In [1]:
import argparse
import logging
import db.mariadb as mariadb
import os
import MySQLdb

import imageio
import numpy as np

import keras
from keras.preprocessing import image

from keras.applications.resnet50 import preprocess_input
from requests_futures.sessions import FuturesSession
from PIL.Image import DecompressionBombError

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logging.basicConfig(format='%(asctime)s-%(levelname)s-%(name)s - %(message)s')


def load_model():
    model = keras.applications.resnet50.ResNet50(include_top=False, pooling='avg')
    # model.summary()
    return model


model = load_model()


Using TensorFlow backend.


In [2]:
# Change according to your database setup (these are the defaults, see .env file)

con = mariadb.get_connection("127.0.0.1", 3308, "image_processing", "user", "user_pw")

files_batch = mariadb.get_files_data(0, 800000, con)

con.close()

# Use every 100th image of the 800000
files_batch = files_batch[::100]
features_train = []
features_test = []

features_train_info = []
features_test_info = []

# Create training and test features
counter = 1
for (img_id, name, path, url) in files_batch:
    try:
        img = image.load_img(path, target_size=(224, 224))
        # print(type(img))
        img_data = image.img_to_array(img)
        # print(type(img_data))
        # print(img_data.shape)
        img_data = np.expand_dims(img_data, axis=0)
        # print(img_data.shape)
        img_data = preprocess_input(img_data)
        # print(img_data.shape)
        res_net_feature = model.predict(img_data)
        # print(res_net_feature.shape)
        res_net_feature = np.array(res_net_feature).flatten()
        # print(res_net_feature.shape)

        if counter % 3 == 0:
            features_test.append(res_net_feature)
            features_test_info.append((img_id, name, path, url))
        else:
            features_train.append(res_net_feature)
            features_train_info.append((img_id, name, path, url))

        counter += 1

    except OSError as e:
        print(e)
    except DecompressionBombError as e:
        print(e)
        print(path)

features_train = np.array(features_train)
features_test = np.array(features_test)

print(features_train.shape)
print(len(features_train_info))

print(features_test.shape)
print(len(features_test_info))

cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/FADatenbankabb0700c/oA_14610.jpg'
cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/FADatenbankabb0700b/FA-Kae54-22_002921808,02.jpg'
cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/FADatenbankabb0700d/FA996-12_0002921601,11.jpg'
cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/FADatenbankabb0700d/Mal1043-01_21751,03.jpg'
cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/FADatenbankabb0700d/Mal18-07_140891,01.jpg'


/usr/lib/python3.7/site-packages/PIL/Image.py:2600: DecompressionBombWarning: Image size (131627440 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/usr/lib/python3.7/site-packages/PIL/Image.py:2600: DecompressionBombWarning: Image size (96499176 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/usr/lib/python3.7/site-packages/PIL/Image.py:2600: DecompressionBombWarning: Image size (129393132 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/FADatenbankabb0575/D-DAI-ROM-56.927_002922407.jpg'
cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/FADatenbankabb0577/D-DAI-ROM-67.16_56977.jpg'
cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/FADatenbankabb0577/D-DAI-ROM-71.624_000300141352.jpg'


/usr/lib/python3.7/site-packages/PIL/Image.py:2600: DecompressionBombWarning: Image size (100186268 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


Image size (205393290 pixels) exceeds limit of 178956970 pixels, could be decompression bomb DOS attack.


/usr/lib/python3.7/site-packages/PIL/Image.py:2600: DecompressionBombWarning: Image size (163745498 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/usr/lib/python3.7/site-packages/PIL/Image.py:2600: DecompressionBombWarning: Image size (127422594 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0067.JPG'
cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0175.JPG'
cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0277.JPG'
cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0376.JPG'
cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0594.JPG'
cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0694.JPG'
cannot identify image file '/home/shohl/Bilder/idai_cloud_mount/dai-rom-fotothek-2003/Bestand-D-DAI-ROM-2003.0794.JPG'
(5323, 2048)
5323
(2661, 2048)
2661


In [3]:
print(np.max(features_train))
print(np.max(features_test))

# TODO: Evaluate which set has higher max value and select higher
features_train_scaled = features_train / np.max(features_train)
features_test_scaled = features_test / np.max(features_train)

print(np.max(features_train_scaled))
print(np.max(features_test_scaled))

29.489676
21.073988


In [4]:
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=10)

result = neigh.fit(features_train_scaled)

neighbours = result.kneighbors()

In [5]:
print(len(neighbours))
print(len(neighbours[0]))
print(neighbours[0])
print(len(neighbours[1]))
print(neighbours[1])

2
5323
[[0.8869949  0.95451961 0.96011991 ... 0.9791909  0.98546634 0.9955557 ]
 [1.26883223 1.30483152 1.3814718  ... 1.44745823 1.44969341 1.45190205]
 [1.1523444  1.19131711 1.20262143 ... 1.2408876  1.2468743  1.2603429 ]
 ...
 [1.03671313 1.03765333 1.06060254 ... 1.08386614 1.08407201 1.08443034]
 [1.06930514 1.07236736 1.08549986 ... 1.11641227 1.11959069 1.12072677]
 [0.74386297 0.77322731 0.79062213 ... 0.83160336 0.83692799 0.84112598]]
5323
[[1821 2854 2761 ... 3762 2723 2719]
 [ 721 2822 3091 ...  208 4134 1022]
 [ 184  204 1096 ... 1089 2822 3232]
 ...
 [4274 2145 4962 ... 1395 2131 3367]
 [5193 1383 4025 ... 1060 4137 1394]
 [5192 5314  160 ... 4712 5181 1056]]


In [36]:
# Initialize and run autoencoder

from keras.models import Sequential, Model
from keras.layers import Dense
from keras.optimizers import Adam, Adadelta
from keras import regularizers
import keras.backend as K

def euclidean_distance_loss(y_true, y_pred):
    """
    Euclidean distance loss
    https://en.wikipedia.org/wiki/Euclidean_distance
    :param y_true: TensorFlow/Theano tensor
    :param y_pred: TensorFlow/Theano tensor of the same shape as y_true
    :return: float
    """
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))


m = Sequential()

m.add(Dense(512, activation='elu', input_shape=(features_train_scaled.shape[1],)))
m.add(Dense(32, activation='linear', name="bottleneck"))
m.add(Dense(512, activation='elu'))
m.add(Dense(features_train.shape[1], activation='sigmoid'))

m.compile(loss=euclidean_distance_loss, optimizer=Adadelta())

history = m.fit(features_train_scaled, features_train_scaled, batch_size=128, epochs=300, verbose=1,
                validation_data=(features_test_scaled, features_test_scaled))

encoder = Model(m.input, m.get_layer('bottleneck').output)
enc = encoder.predict(features_train_scaled) # returns the encoded values (32 floats instead of 2048)

Train on 5323 samples, validate on 2661 samples
Epoch 1/300
5323/5323 [==============================] - 5s 977us/step - loss: 5.4996 - val_loss: 1.1866
Epoch 2/300
5323/5323 [==============================] - 4s 741us/step - loss: 1.1657 - val_loss: 1.1453
Epoch 3/300
5323/5323 [==============================] - 4s 745us/step - loss: 1.1461 - val_loss: 1.1506
Epoch 4/300
5323/5323 [==============================] - 4s 776us/step - loss: 1.1339 - val_loss: 1.1335
Epoch 5/300
5323/5323 [==============================] - 4s 737us/step - loss: 1.1252 - val_loss: 1.1236
Epoch 6/300
5323/5323 [==============================] - 4s 726us/step - loss: 1.1186 - val_loss: 1.1103
Epoch 7/300
5323/5323 [==============================] - 4s 724us/step - loss: 1.1107 - val_loss: 1.1112
Epoch 8/300
5323/5323 [==============================] - 4s 726us/step - loss: 1.1074 - val_loss: 1.1174
Epoch 9/300
5323/5323 [==============================] - 4s 728us/step - loss: 1.1042 - val_loss: 1.1040
Epoch 1

Epoch 78/300
5323/5323 [==============================] - 4s 720us/step - loss: 0.8252 - val_loss: 0.8182
Epoch 79/300
5323/5323 [==============================] - 4s 717us/step - loss: 0.8212 - val_loss: 0.8208
Epoch 80/300
5323/5323 [==============================] - 4s 723us/step - loss: 0.8224 - val_loss: 0.8153
Epoch 81/300
5323/5323 [==============================] - 4s 718us/step - loss: 0.8189 - val_loss: 0.8111
Epoch 82/300
5323/5323 [==============================] - 4s 720us/step - loss: 0.8166 - val_loss: 0.8275
Epoch 83/300
5323/5323 [==============================] - 4s 716us/step - loss: 0.8144 - val_loss: 0.8122
Epoch 84/300
5323/5323 [==============================] - 4s 716us/step - loss: 0.8126 - val_loss: 0.8146
Epoch 85/300
5323/5323 [==============================] - 4s 718us/step - loss: 0.8098 - val_loss: 0.8131
Epoch 86/300
5323/5323 [==============================] - 4s 716us/step - loss: 0.8087 - val_loss: 0.8084
Epoch 87/300
5323/5323 [======================

Epoch 155/300
5323/5323 [==============================] - 4s 724us/step - loss: 0.7023 - val_loss: 0.7075
Epoch 156/300
5323/5323 [==============================] - 4s 720us/step - loss: 0.7002 - val_loss: 0.7063
Epoch 157/300
5323/5323 [==============================] - 4s 722us/step - loss: 0.7005 - val_loss: 0.7109
Epoch 158/300
5323/5323 [==============================] - 4s 722us/step - loss: 0.7001 - val_loss: 0.7035
Epoch 159/300
5323/5323 [==============================] - 4s 721us/step - loss: 0.6966 - val_loss: 0.7116
Epoch 160/300
5323/5323 [==============================] - 4s 720us/step - loss: 0.6988 - val_loss: 0.7074
Epoch 161/300
5323/5323 [==============================] - 4s 718us/step - loss: 0.6956 - val_loss: 0.7054
Epoch 162/300
5323/5323 [==============================] - 4s 722us/step - loss: 0.6955 - val_loss: 0.7065
Epoch 163/300
5323/5323 [==============================] - 4s 722us/step - loss: 0.6940 - val_loss: 0.7031
Epoch 164/300
5323/5323 [============

5323/5323 [==============================] - 4s 718us/step - loss: 0.6379 - val_loss: 0.6675
Epoch 232/300
5323/5323 [==============================] - 4s 714us/step - loss: 0.6381 - val_loss: 0.6652
Epoch 233/300
5323/5323 [==============================] - 4s 715us/step - loss: 0.6378 - val_loss: 0.6575
Epoch 234/300
5323/5323 [==============================] - 4s 714us/step - loss: 0.6357 - val_loss: 0.6590
Epoch 235/300
5323/5323 [==============================] - 4s 714us/step - loss: 0.6352 - val_loss: 0.6616
Epoch 236/300
5323/5323 [==============================] - 4s 717us/step - loss: 0.6352 - val_loss: 0.6604
Epoch 237/300
5323/5323 [==============================] - 4s 717us/step - loss: 0.6336 - val_loss: 0.6553
Epoch 238/300
5323/5323 [==============================] - 4s 714us/step - loss: 0.6334 - val_loss: 0.6586
Epoch 239/300
5323/5323 [==============================] - 4s 715us/step - loss: 0.6332 - val_loss: 0.6541
Epoch 240/300
5323/5323 [==========================

In [37]:
result_enc = neigh.fit(enc)
neighbours_enc = result_enc.kneighbors()

In [38]:
enc_r = m.predict(features_train_scaled)      # reconstruction, 2048 -> 32 -> 2048

In [39]:
result_enc_r = neigh.fit(enc_r)
neighbours_enc_r = result_enc.kneighbors()

In [40]:
np.min(features_train_scaled)

0.0

In [41]:
np.min(features_test_scaled)

0.0

In [42]:
np.max(features_train_scaled)

1.0

In [43]:
np.max(features_test_scaled)

0.71462256

In [44]:
print(neighbours[1][0])
print(neighbours_enc_r[1][0])
print(neighbours_enc[1][0])

[1821 2854 2761 3123 3441 1823 3599 3762 2723 2719]
[4093 2854 2723 4018 3123 2739 3762 4224 1821 2642]
[4093 2854 2642 2723 3762 1821 5084 4012 4018 2742]


In [45]:
print(neighbours[1][1])
print(neighbours_enc_r[1][1])
print(neighbours_enc[1][1])

[ 721 2822 3091 4785 4879 2877 1107  208 4134 1022]
[ 721 2822  148 1092 1365  150  204    2 1842 1088]
[ 721 2822  150    2  173  148 1842 1088 1092  204]


In [46]:
print(neighbours[1][2])
print(neighbours_enc_r[1][2])
print(neighbours_enc[1][2])

[ 184  204 1096  225 1090 1088 1085 1089 2822 3232]
[ 204  184 1090  225 1107 1088 1089 1092  721 3232]
[ 204 1090  225  184 1088 1089 1092 1107 1085 1096]


In [47]:
enc.shape

(5323, 32)

In [48]:
features_train_scaled.shape

(5323, 2048)

In [49]:
np.save('features.npy', features_train_scaled)
np.save('alternate_features_compresssed.npy', enc)
np.save('alternate_featuers_reconstructed.npy', enc_r)

In [50]:
np.save('features_test.npy', features_test_scaled)

In [51]:
import json


with open('features_train_info.txt', 'w') as outfile:  
    json.dump(features_train_info, outfile)
    
with open('features_test_info.txt', 'w') as outfile:  
    json.dump(features_test_info, outfile)

In [52]:
from keras.models import load_model

encoder.save('encoder.h5')
m.save('autoencoder.h5')